In [2]:
import sys
import os

import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import plotly.graph_objects as go # Nick did this, idk if it's standard but it's cute cause then you get go.Figure()
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

In [3]:
def get_file_data(file_paths) -> pd.DataFrame:
    metric_data = pd.DataFrame()

    for path in file_paths:
        # Import the data
        metric_datum = pd.read_csv(path, skiprows=1, sep='\t', low_memory=False)

        # Rename columns to have value and units, and remove the previous row for units
        metric_datum.rename(columns={col: f"{col} {metric_datum[col][0]}" for col in metric_datum.columns}, inplace=True)
        metric_datum = metric_datum.drop(0)

        # Convert all values to floats
        metric_datum = metric_datum.apply(pd.to_numeric, errors='coerce')

        # Drop any rows with NaN values (if necessary)
        metric_datum = metric_datum.dropna()

        # Reset index
        metric_datum.reset_index(drop=True, inplace=True)

        metric_data = pd.concat([metric_data, metric_datum])
        print(len(metric_datum))

    print(f"Loaded {len(metric_data)} columns.")

    return metric_data

def get_run_data(selected_runs) -> pd.DataFrame:
    file_names = [f'./RunData_Cornering_ASCII_SI_Round9/B2356run{rn}.dat' for rn in selected_runs]
    return get_file_data(file_names)

def get_all_raw_data() -> pd.DataFrame:
    data_files = []
    
    for root, _, files in os.walk("RawData_Cornering_ASCII_SI_Round9_Runs1to15 (1)"):
        for filename in files:
            data_files.append(os.path.join(root, filename))
    
    return get_file_data(data_files)

def get_all_run_data() -> pd.DataFrame:
    data_files = []
    
    for root, _, files in os.walk("RunData_Cornering_ASCII_SI_Round9"):
        for filename in files:
            data_files.append(os.path.join(root, filename))
    
    return get_file_data(data_files)


In [5]:
SELECTED_RUNS = [4, 5, 6, 8, 9]
# SELECTED_RUNS = [2] + list(range(4, 26)) + list(range(27, 45)) + [46, 49]

all_data = get_run_data(SELECTED_RUNS)
# all_data = get_all_raw_data()
# all_data = get_all_run_data()

36194
31138
52331
61020
52322
Loaded 233005 columns.


In [23]:
from scipy.interpolate import LinearNDInterpolator

points = all_data[["SA deg", "MZ Nm"]]
values = all_data["FY N"]

interp = LinearNDInterpolator(points, values)

print(interp)

In [ ]:
CA_YELLOW = "#FDB515"
BK_BLUE = "#002676"

LABEL_COLOR = BK_BLUE
TICK_COLOR = CA_YELLOW

SCENE_BACKGROUND = "white" #"Greenscreen", to photoshop out background
PLANE_COLOR = "#242424"

Fz_domain = np.linspace(all_data["SA deg"].min(), all_data["SA deg"].max(), 100, endpoint=True)[1:-1]
Mz_domain = np.linspace(all_data["MZ Nm"].min(), all_data["MZ Nm"].max(), 100, endpoint=True)[1:-1]

Fz_grid, Mz_grid = np.meshgrid(Fz_domain, Mz_domain)

print(Fz_grid, Mz_grid)

Fy_values = interp(Fz_grid, Mz_grid)

# fig = go.Figure(data=[go.Surface(
#         x=Fz_domain,
#         y=Mz_domain,
#         z=Fy_values
#     )])

# all_data = all_data.sample(10000)
# reduced_data = all_data[all_data["FY N"] > 0].sample(50000)
reduced_data = all_data.sample(50000)

fig = go.Figure(data=[
    go.Scatter(
        x=reduced_data["MZ Nm"],
        y=reduced_data["FY N"],
        mode='markers',
        marker=dict(
            color=reduced_data["SA deg"],
            colorscale='Viridis',  # You can change to 'Jet', 'Cividis', etc.
            colorbar=dict(
                title='Slip Angle (deg)',
                # titlefont=dict(color=LABEL_COLOR, size=20),
                # tickfont=dict(color=LABEL_COLOR, size=14),
            ),
            showscale=True,
        )
    )
])

def linear_fit(mz, a, b):
    return a * mz + b

popt, _ = curve_fit(linear_fit, reduced_data["MZ Nm"], reduced_data["FY N"])

fig.add_trace(
            go.Scatter(
                x=np.linspace(-50,50,100), 
                y=linear_fit(Mz_domain, -25, 1000), 
                mode="lines", 
                # marker=dict(
                #     color=BIN_COLORS[i]
                #     ), 
                # name=f'{(Fz_bins[i][0] + Fz_bins[i][1]) // 2}',
                legendrank=100 # try a high value to prioritize
            )
        )

fig.update_layout(
    xaxis=dict(
        title=dict(text='Aligning Torque (Nm)', font=dict(color=LABEL_COLOR, size=30)),
        tickfont=dict(color=LABEL_COLOR, size=16),
    ),
    yaxis=dict(
        title=dict(text='Lateral Force (N)', font=dict(color=LABEL_COLOR, size=30)),
        tickfont=dict(color=LABEL_COLOR, size=16),
    ),
    plot_bgcolor=SCENE_BACKGROUND,
    title=dict(
        x=0.5,
        xanchor='center',
        text='Lateral Force vs Aligning Torque (colored by Slip Angle)',
        font=dict(size=36),
    )
)

fig.write_html("mz_out/figures/load_force_mz.html")


[[-11.7859899 -11.5429798 -11.2999697 ...  11.2999697  11.5429798
   11.7859899]
 [-11.7859899 -11.5429798 -11.2999697 ...  11.2999697  11.5429798
   11.7859899]
 [-11.7859899 -11.5429798 -11.2999697 ...  11.2999697  11.5429798
   11.7859899]
 ...
 [-11.7859899 -11.5429798 -11.2999697 ...  11.2999697  11.5429798
   11.7859899]
 [-11.7859899 -11.5429798 -11.2999697 ...  11.2999697  11.5429798
   11.7859899]
 [-11.7859899 -11.5429798 -11.2999697 ...  11.2999697  11.5429798
   11.7859899]] [[-121.08444444 -121.08444444 -121.08444444 ... -121.08444444
  -121.08444444 -121.08444444]
 [-118.39888889 -118.39888889 -118.39888889 ... -118.39888889
  -118.39888889 -118.39888889]
 [-115.71333333 -115.71333333 -115.71333333 ... -115.71333333
  -115.71333333 -115.71333333]
 ...
 [ 134.04333333  134.04333333  134.04333333 ...  134.04333333
   134.04333333  134.04333333]
 [ 136.72888889  136.72888889  136.72888889 ...  136.72888889
   136.72888889  136.72888889]
 [ 139.41444444  139.41444444  139.414

In [ ]:
CA_YELLOW = "#FDB515"
BK_BLUE = "#002676"

LABEL_COLOR = BK_BLUE
TICK_COLOR = CA_YELLOW

SCENE_BACKGROUND = "white" #"Greenscreen", to photoshop out background
PLANE_COLOR = "#242424"

reduced_data = all_data.sample(50000)

fig = go.Figure(data = [
    # go.Scatter3d(
    #     x=reduced_data["FZ N"],
    #     y=reduced_data["MZ Nm"],
    #     z=reduced_data["FY N"],
    #     mode='markers'
    # )
])

Fz_domain = np.linspace(reduced_data["FZ N"].min(), reduced_data["FZ N"].max(), 100)[1:-1]
Mz_domain = np.linspace(reduced_data["MZ Nm"].min(), reduced_data["MZ Nm"].max(), 100)[1:-1]

FZ, MZ = np.meshgrid(Fz_domain, Mz_domain)

Fy_surface = (-10/1500 * FZ) * MZ  # now Fy_surface is 2D and matches FZ and MZ

fig.add_trace(
    go.Surface(
        x=FZ,
        y=MZ,
        z=Fy_surface,
        colorscale="Viridis",
        # opacity=0.5,
        showscale=False
    )
)

# fig.add_trace(go.Scatter3d(
#     x=[-1500] * len(reduced_data),
#     y=reduced_data["MZ Nm"],
#     z=10 * reduced_data["MZ Nm"]
# ))

fig.update_layout(
    scene=dict(
        bgcolor=SCENE_BACKGROUND,
        xaxis=dict(
            gridcolor=TICK_COLOR,
            title=dict(text='Load Force (N)', font=dict(color=LABEL_COLOR, size=30)),
            tickfont=dict(color=LABEL_COLOR),
            showbackground=True,
            backgroundcolor=PLANE_COLOR
        ),
        yaxis=dict(
            gridcolor=TICK_COLOR,
            title=dict(text='Aligning Torque (Nm)', font=dict(color=LABEL_COLOR, size=30)),
            tickfont=dict(color=LABEL_COLOR),
            showbackground=True,
            backgroundcolor=PLANE_COLOR
        ),
        zaxis=dict(
            gridcolor=TICK_COLOR,
            title=dict(text='Lateral Force (N)', font=dict(color=LABEL_COLOR, size=30)),
            tickfont=dict(color=LABEL_COLOR),
            showbackground=True,
            backgroundcolor=PLANE_COLOR
        ),
    ),
    title=dict(
        x=0.5,
        xanchor='center',
        text='Lateral Force vs Load Force and Aligning Torque',
        font=dict(
            size=36,
        )
    )
)

fig.write_html("mz_out/figures/3d_plot.html")

In [4]:
def bin_data(dataset : pd.DataFrame, var : str, bins):
    binned_sets = []
    for bin_range in bins:
        binned_sets.append(dataset[(dataset[var] >= bin_range[0]) & (dataset[var] < bin_range[1])])

    print([len(bin_i) for bin_i in binned_sets])

    return binned_sets  

In [5]:
def pacejka_model(alpha, B, C, D, E):
    return D * np.sin(C * np.arctan(B * alpha - E * (B * alpha - np.arctan(B * alpha))))

Reset Data

In [ ]:
popt = np.zeros(shape=(5, 4))

Fz_bins = [(-1300, -1000), (-1000, -750), (-750, -500), (-500, -300), (-300, -150)]
PACEJKA_PARAMS_GUESS = [-0.1, 0.1, 2000, 0.3]  # Update as needed

all_data_binned = bin_data(all_data, "FZ N", Fz_bins)

[58320, 42996, 43606, 43602, 43586]


Update Individual Params

In [160]:
which_bin = 4
reduced_data = all_data_binned[which_bin]
# reduced_data = reduced_data.sample(50000)

fig = go.Figure(data=[
    go.Scatter(
        x=reduced_data["SA deg"],
        y=reduced_data["MZ Nm"],
        mode='markers',
        marker=dict(
            color=reduced_data["FZ N"],
            colorscale='Viridis',  # You can change to 'Jet', 'Cividis', etc.
            colorbar=dict(
                title='Load Force (N)',
                # titlefont=dict(color=LABEL_COLOR, size=20),
                # tickfont=dict(color=LABEL_COLOR, size=14),
            ),
            showscale=True,
        )
    )
])

i_popt, _ = curve_fit(pacejka_model, all_data_binned[which_bin]["SA deg"], all_data_binned[which_bin]["MZ Nm"], p0=PACEJKA_PARAMS_GUESS)
popt[which_bin] = i_popt

SA_range = np.linspace(all_data_binned[which_bin]["SA deg"].min(), all_data_binned[which_bin]["SA deg"].max(), 100)

fig.add_trace(
    go.Scatter(
        x=SA_range, 
        y=pacejka_model(SA_range, *i_popt), 
        mode="lines", 
        # marker=dict(
        #     color=BIN_COLORS[i]
        #     ), 
        # name=f'{(Fz_bins[i][0] + Fz_bins[i][1]) // 2}',
        legendrank=100 # try a high value to prioritize
    )
)

fig.write_html("mz_out/figures/mz_slip_angle.html")

for i in range(5):
    print(str(popt[i]) + (" <---- Updated" if which_bin == i else ""))

[ -0.15188057   3.05654741 -58.13341104   0.47746707]
[ -0.17208438   3.031611   -41.01566735   0.54934547]
[6.77187350e-01 1.46044022e-02 2.08960691e+03 1.22241973e+00]
[7.22871238e-01 2.20865746e-02 7.70787047e+02 1.21733908e+00]
[-0.48030498  1.96443578 -5.39105738  1.36454797] <---- Updated


In [181]:
CA_YELLOW = "#FDB515"
BK_BLUE = "#002676"

LABEL_COLOR = BK_BLUE
TICK_COLOR = CA_YELLOW

SCENE_BACKGROUND = "white" #"Greenscreen", to photoshop out background
PLANE_COLOR = "#242424"

# SA_max_abs = min(max(abs(all_data_binned[i]["SA deg"])) for i in range(5))
SA_max_abs = 15

SA_range = np.linspace(-SA_max_abs, SA_max_abs, 100)

fig = go.Figure()

for i in range(5):
    fig.add_trace(
        go.Scatter(
            x=SA_range, 
            y=pacejka_model(SA_range, *popt[i]), 
            mode="lines", 
            # marker=dict(
            #     color=BIN_COLORS[i]
            #     ), 
            name=f'{(Fz_bins[i][0] + Fz_bins[i][1]) // 2}',
            legendrank=100 # try a high value to prioritize
        )
    )

fig.update_layout(
    title=dict(
        x=0.5,
        xanchor='center',
        text='Aligning Torque vs Slip Angle',
        font=dict(size=36),
    ),
    xaxis=dict(
        title=dict(text='Slip Angle (deg)', font=dict(color=LABEL_COLOR, size=30)),
        tickfont=dict(color=LABEL_COLOR, size=16),
    ),
    yaxis=dict(
        title=dict(text='Aligning Torque (Nm)', font=dict(color=LABEL_COLOR, size=30)),
        tickfont=dict(color=LABEL_COLOR, size=16),
    ),
    # plot_bgcolor=SCENE_BACKGROUND,
    legend=dict(
        x=0.02,
        y=0.95, 
        font = dict(
            size=16
        )
    ),
    width=1000, 
    height=900,
    legend_title_text='Fz (N)',
    # plot_bgcolor=PLANE_COLOR
)

fig.write_html("mz_out/figures/binned_mz_slip_angle.html")

In [ ]:
reduced_data = all_data.sample(100000)

fig = go.Figure(data=[
    go.Scatter(
        x=reduced_data["SA deg"],
        y=reduced_data["MZ Nm"],
        mode='markers',
        marker=dict(
            color=reduced_data["FZ N"],
            colorscale='Viridis',  # You can change to 'Jet', 'Cividis', etc.
            colorbar=dict(
                title='Load Force (N)',
                # titlefont=dict(color=LABEL_COLOR, size=20),
                # tickfont=dict(color=LABEL_COLOR, size=14),
            ),
            showscale=True,
        )
    )
])

def linear_fit(mz, a, b):
    return a * mz + b

# popt, _ = curve_fit(linear_fit, reduced_data["MZ Nm"], reduced_data["FY N"])

# fig.add_trace(
#             go.Scatter(
#                 x=np.linspace(-50,50,100), 
#                 y=linear_fit(Mz_domain, -25, 1000), 
#                 mode="lines", 
#                 # marker=dict(
#                 #     color=BIN_COLORS[i]
#                 #     ), 
#                 # name=f'{(Fz_bins[i][0] + Fz_bins[i][1]) // 2}',
#                 legendrank=100 # try a high value to prioritize
#             )
#         )

fig.update_layout(
    xaxis=dict(
        title=dict(text='Slip Angle (deg)', font=dict(color=LABEL_COLOR, size=30)),
        tickfont=dict(color=LABEL_COLOR, size=16),
    ),
    yaxis=dict(
        title=dict(text='Aligning Torque (Nm)', font=dict(color=LABEL_COLOR, size=30)),
        tickfont=dict(color=LABEL_COLOR, size=16),
    ),
    plot_bgcolor=SCENE_BACKGROUND,
    title=dict(
        x=0.5,
        xanchor='center',
        text='Lateral Force vs Aligning Torque (colored by Slip Angle)',
        font=dict(size=36),
    )
)

fig.write_html("mz_out/figures/mz_slip_angle.html")


[[-11.7859899 -11.5429798 -11.2999697 ...  11.2999697  11.5429798
   11.7859899]
 [-11.7859899 -11.5429798 -11.2999697 ...  11.2999697  11.5429798
   11.7859899]
 [-11.7859899 -11.5429798 -11.2999697 ...  11.2999697  11.5429798
   11.7859899]
 ...
 [-11.7859899 -11.5429798 -11.2999697 ...  11.2999697  11.5429798
   11.7859899]
 [-11.7859899 -11.5429798 -11.2999697 ...  11.2999697  11.5429798
   11.7859899]
 [-11.7859899 -11.5429798 -11.2999697 ...  11.2999697  11.5429798
   11.7859899]] [[-121.08444444 -121.08444444 -121.08444444 ... -121.08444444
  -121.08444444 -121.08444444]
 [-118.39888889 -118.39888889 -118.39888889 ... -118.39888889
  -118.39888889 -118.39888889]
 [-115.71333333 -115.71333333 -115.71333333 ... -115.71333333
  -115.71333333 -115.71333333]
 ...
 [ 134.04333333  134.04333333  134.04333333 ...  134.04333333
   134.04333333  134.04333333]
 [ 136.72888889  136.72888889  136.72888889 ...  136.72888889
   136.72888889  136.72888889]
 [ 139.41444444  139.41444444  139.414

In [7]:
# SELECTED_RUNS = [4, 5, 6, 8, 9]
# SELECTED_RUNS = [2] + list(range(4, 26)) + list(range(27, 45)) + [46, 49]

# all_data = get_run_data(SELECTED_RUNS)
all_data = get_all_raw_data()
# all_data = get_all_run_data()

79851
210528
220043
79849
210550
220034
79861
127192
101287
220017
79867
210565
220001
Loaded 2059645 columns.


In [8]:
reduced_data = all_data.sample(100000)

fig = go.Figure(data=[
    go.Scatter(
        x=reduced_data["SA deg"],
        y=reduced_data["FY N"],
        mode='markers',
        marker=dict(
            color=reduced_data["IA deg"],
            colorscale='Viridis',  # You can change to 'Jet', 'Cividis', etc.
            colorbar=dict(
                title='Camber (deg)',
                # titlefont=dict(color=LABEL_COLOR, size=20),
                # tickfont=dict(color=LABEL_COLOR, size=14),
            ),
            showscale=True,
        )
    )
])

# i_popt, _ = curve_fit(pacejka_model, all_data_binned[which_bin]["SA deg"], all_data_binned[which_bin]["MZ Nm"], p0=PACEJKA_PARAMS_GUESS)
# popt[which_bin] = i_popt

# SA_range = np.linspace(all_data_binned[which_bin]["SA deg"].min(), all_data_binned[which_bin]["SA deg"].max(), 100)

# fig.add_trace(
#     go.Scatter(
#         x=SA_range, 
#         y=pacejka_model(SA_range, *i_popt), 
#         mode="lines", 
#         # marker=dict(
#         #     color=BIN_COLORS[i]
#         #     ), 
#         # name=f'{(Fz_bins[i][0] + Fz_bins[i][1]) // 2}',
#         legendrank=100 # try a high value to prioritize
#     )
# )

fig.write_html("camber_out/figures/camber_slip_angle.html")

# for i in range(5):
#     print(str(popt[i]) + (" <---- Updated" if which_bin == i else ""))

In [9]:
bins = [(-4.5, -3.5), (-3.5, -2.5), (-2.5, -1.5), (-1.5, -0.5), (-0.5, 0.5), (0.5, 1.5), (1.5, 2.5), (2.5, 3.5), (3.5, 4.5)]
# bins = [(-0.5, 0.5), (1.5, 2.5), (3.5, 4.5)]
PACEJKA_PARAMS_GUESS = [-0.1, 0.1, 2000, 0.3]  # Update as needed

popt = np.zeros(shape=(len(bins), len(PACEJKA_PARAMS_GUESS)))

all_data_binned = bin_data(all_data, "IA deg", bins)

[11640, 22364, 22444, 22528, 1053808, 25183, 448701, 25044, 427372]


In [17]:
which_bin = 8
reduced_data = all_data_binned[which_bin]
# reduced_data = reduced_data.sample(50000)

fig = go.Figure(data=[
    go.Scatter(
        x=reduced_data["SA deg"],
        y=reduced_data["FY N"],
        mode='markers',
        marker=dict(
            color=reduced_data["IA deg"],
            colorscale='Viridis',  # You can change to 'Jet', 'Cividis', etc.
            colorbar=dict(
                title='Camber (deg)',
                # titlefont=dict(color=LABEL_COLOR, size=20),
                # tickfont=dict(color=LABEL_COLOR, size=14),
            ),
            showscale=True,
        )
    )
])

i_popt, _ = curve_fit(pacejka_model, all_data_binned[which_bin]["SA deg"], all_data_binned[which_bin]["FY N"], p0=PACEJKA_PARAMS_GUESS)
popt[which_bin] = i_popt

SA_range = np.linspace(all_data_binned[which_bin]["SA deg"].min(), all_data_binned[which_bin]["SA deg"].max(), 100)

fig.add_trace(
    go.Scatter(
        x=SA_range, 
        y=pacejka_model(SA_range, *i_popt), 
        mode="lines", 
        # marker=dict(
        #     color=BIN_COLORS[i]
        #     ), 
        # name=f'{(Fz_bins[i][0] + Fz_bins[i][1]) // 2}',
        legendrank=100 # try a high value to prioritize
    )
)

fig.write_html("camber_out/figures/camber_slip_angle.html")

for i in range(len(bins)):
    print(str(popt[i]) + (" <---- Updated" if which_bin == i else ""))

[-7.79161117e-02  1.77271622e+00  2.46446338e+03 -1.39932016e+01]
[-3.60692467e-01  5.70230755e-02  3.28018737e+04 -1.00937549e+00]
[-8.78648115e-02  2.40285875e+00  2.53141252e+03  3.00639371e-01]
[-1.62894068e-01  1.52120320e+00  2.56929367e+03  3.05899738e-01]
[0. 0. 0. 0.]
[-1.26961120e-01  1.81216366e+00  2.44533829e+03  3.05193809e-01]
[-1.49402507e-01  1.68186521e+00  1.57210411e+03  3.53804042e-01]
[-1.62594940e-01  1.08835062e+00  2.47975605e+03 -2.58535083e+00]
[-1.74171972e-01  1.44501395e+00  1.50568176e+03 -1.29409880e-01] <---- Updated


Coefficient Plot

In [1]:
PACEJKA_PARAMS_NAMES = ["B", "C", "D", "E"]

def coefficient_figs(params_binned, IA_bins):
    fig = make_subplots(rows=2, cols=3, subplot_titles=[f"{PACEJKA_PARAMS_NAMES[i]} value over camber bins" for i in range(len(PACEJKA_PARAMS_NAMES))])

    for i in range(len(PACEJKA_PARAMS_NAMES)):
        values_for_this_param = params_binned[:,i] # ith column
        IA_medians = [(ub + lb) / 2 for lb, ub in IA_bins]

        fig.add_trace(go.Scatter(x=IA_medians, y=values_for_this_param), row = i // 3 + 1, col = i % 3 + 1)
        fig.update_xaxes(title_text=f"IA Bin Median", row = i // 3 + 1, col = i % 3 + 1)
        fig.update_yaxes(title_text=f"Value of param {PACEJKA_PARAMS_NAMES[i]}", row = i // 3 + 1, col = i % 3 + 1)

        # coefficients, _ = curve_fit(PACEJKA_PARAM_FIT_FNS[i], Fz_medians, values_for_this_param, maxfev=150000, p0=PACEJKA_PARAM_FIT_GUESS[i])
        # Fz_domain = np.linspace(Fz_medians[0], Fz_medians[-1], 50)
        # fig.add_trace(go.Scatter(x=Fz_domain, y=PACEJKA_PARAM_FIT_FNS[i](Fz_domain, *coefficients)), row = i // 3 + 1, col = i % 3 + 1)

        # print(f"Pacejka params for {PACEJKA_PARAMS_NAMES[i]}:", coefficients)
        # print(fit_print(PACEJKA_PARAM_FIT_FNS[i], *coefficients))

    fig.update_layout(title_text=f"Values for Pacejka params over IA {SELECTED_RUNS}")

    return fig

coefficient_figs(np.delete(popt, 4, axis=0), np.delete(bins, 4, axis=0)).write_html("camber_out/figures/coefficient_fig.html")

NameError: name 'np' is not defined

In [22]:
CA_YELLOW = "#FDB515"
BK_BLUE = "#002676"

LABEL_COLOR = BK_BLUE
TICK_COLOR = CA_YELLOW

SCENE_BACKGROUND = "white" #"Greenscreen", to photoshop out background
PLANE_COLOR = "#242424"

# SA_max_abs = min(max(abs(all_data_binned[i]["SA deg"])) for i in range(5))
SA_max_abs = 9

SA_range = np.linspace(-SA_max_abs, SA_max_abs, 100)

fig = go.Figure()

for i in range(len(bins)):
    if i == 4 : continue
    fig.add_trace(
        go.Scatter(
            x=SA_range, 
            y=pacejka_model(SA_range, *popt[i]), 
            mode="lines", 
            # marker=dict(
            #     color=BIN_COLORS[i]
            #     ), 
            name=f'{(bins[i][0] + bins[i][1]) // 2}',
            legendrank=100 # try a high value to prioritize
        )
    )

fig.update_layout(
    title=dict(
        x=0.5,
        xanchor='center',
        text='Lateral Force vs Slip Angle',
        font=dict(size=36),
    ),
    xaxis=dict(
        title=dict(text='Slip Angle (deg)', font=dict(color=LABEL_COLOR, size=30)),
        tickfont=dict(color=LABEL_COLOR, size=16),
    ),
    yaxis=dict(
        title=dict(text='Lateral Force (N)', font=dict(color=LABEL_COLOR, size=30)),
        tickfont=dict(color=LABEL_COLOR, size=16),
    ),
    # plot_bgcolor=SCENE_BACKGROUND,
    legend=dict(
        x=0.85,
        y=0.97, 
        font = dict(
            size=16
        )
    ),
    width=1000, 
    height=900,
    legend_title_text='IA (deg)',
    # plot_bgcolor=PLANE_COLOR
)

fig.write_html("camber_out/figures/binned_camber_slip_angle.html")

In [ ]:
hyper_coefficients = []
    
for i in range(5):
    values_for_this_param = params_binned[:,i] # ith column
    Fz_medians = [(ub + lb) / 2 for lb, ub in Fz_bins]

    coefficients, _ = curve_fit(PACEJKA_PARAM_FIT_FNS[i], Fz_medians, values_for_this_param, maxfev=150000, p0=PACEJKA_PARAM_FIT_GUESS[i])
    hyper_coefficients.append(coefficients)

# Define the Pacejka Magic Formula for lateral force (FY) with five coefficients
def mvar_pacejka(alpha, Fz):
    B, C, D, E, F = [PACEJKA_PARAM_FIT_FNS[i](Fz, *hyper_coefficients[i]) for i in range(5)]

    return D * np.sin(C * np.arctan(B * alpha - E * (B * alpha - np.arctan(B * alpha))) + F)

Fz_domain = np.linspace(all_data["FZ N"].min(), -225, 100)
SA_domain = np.linspace(all_data["SA deg"].min(), all_data["SA deg"].max(), 100)

Fy_surface = np.zeros(shape=(len(Fz_domain), len(SA_domain)))
for i in range(len(Fz_domain)):
    for j in range(len(SA_domain)):
        Fy_surface[j][i] = mvar_pacejka(SA_domain[j], Fz_domain[i])

fig = go.Figure(data=[go.Surface(
    x=Fz_domain,
    y=SA_domain,
    z=Fy_surface
)])

CA_YELLOW = "#FDB515"
BK_BLUE = "#002676"

LABEL_COLOR = BK_BLUE
TICK_COLOR = CA_YELLOW

SCENE_BACKGROUND = "white" #"Greenscreen", to photoshop out background
PLANE_COLOR = "#242424"

fig.update_layout(
    scene=dict(
        bgcolor=SCENE_BACKGROUND,
        xaxis=dict(
            gridcolor=TICK_COLOR,
            title=dict(text='Load Force (N)', font=dict(color=LABEL_COLOR, size=30)),
            tickfont=dict(color=LABEL_COLOR, size=22),
            showbackground=True,
            backgroundcolor=PLANE_COLOR
        ),
        yaxis=dict(
            gridcolor=TICK_COLOR,
            title=dict(text='Slip Angle (deg)', font=dict(color=LABEL_COLOR, size=30)),
            tickfont=dict(color=LABEL_COLOR, size=22),
            showbackground=True,
            backgroundcolor=PLANE_COLOR
        ),
        zaxis=dict(
            gridcolor=TICK_COLOR,
            title=dict(text='Lateral Force (N)', font=dict(color=LABEL_COLOR, size=30)),
            tickfont=dict(color=LABEL_COLOR, size=22),
            showbackground=True,
            backgroundcolor=PLANE_COLOR
        ),
    ),
    title=dict(
        x=0.5,
        xanchor='center',
        text='Lateral Force vs Load Force and Slip Angle',
        font=dict(
            size=36,
        )
    )
)